# Boy through time

This notebook can be used to temporally average images of an aging person, similar to what was done for miykael's [Noah ages](https://github.com/miykael/noah_ages) project. In short, we use the [dlib](http://dlib.net/) toolbox to detect, extract and align faces from all images. The process to do so is a multi-step process:

1. We use `hog_detector` detector to find the faces in all images. This detector is 'ok-ish' and has the advantage of running very quickly.
2. After running the first face detector, manual inspection of all aligned images needs to be performed and faulty/misaligned images need to be deleted.
3. For all images that were faulty or where the face wasn't detect, we now use the `cnn_face_detection_model_v1` routine to hopefully correct that. This routine is much slower but more accurate and capable.
4. Another manual inspection needs to be performed to remove faulty and misaligned images.
5. Images where both face detector algorithms failed, manual alignmed needs to be performed. Furthermore, dates of missing images can/should be filled up to have one image for every day.

**Side note** about how it works: Dlib's face detection is usually used to extract small 'chips'/patches of pixels that only contain the face. In this case however we decided to keep the full image, but just profit from dlib's routine of aligning the faces according to the 5 landmarks (two eyes, nose and two corners of the mouth). During this procedure, images can also be rescaled to any resolution.

## 1. Setup environment

Before we can start we need to make sure that all relevant packages are installed. We can do this using `pip`.

In [ ]:
!pip install -Uq  dlib~=19.24 matplotlib~=3.8 numpy~=1.26 opencv-python~=4.9 pathlib~=1.0 Pillow~=10.0 scikit-image~=0.22 tqdm~=4.66 pandas

Collect relevant folders and files from github repository. This is only needed when you don't run the notebook from within the repo, e.g. running it via colab.

In [ ]:
# Get github repository data - needed for when you run it via google's colab
!git clone https://github.com/miykael/boy_ages.git

## 1.1. Import relevant packages

In [ ]:
from pathlib import Path
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shutil
from skimage import io, transform
from glob import glob
from matplotlib import patches

from tqdm.notebook import tqdm
import dlib
from PIL import Image, ImageDraw, ImageFont

## 1.2. Specify relevant parameters

In [ ]:
# Filename structure all images have to have to be considered
file_identifier = "PXL_*.jpg"

# Define size of face chip (maximum width or height of raw image)
max_size = 4320

# Downsample factor for face finding
downsample = 2

## 1.3. Establish paths to images

The code in this notebook should run slightly different, depending on if you are running this notebook in google colab or not. So let's detect this:

In [ ]:
try:
    import google.colab

    IN_COLAB = True
except:
    IN_COLAB = False

In [ ]:
if IN_COLAB:
    # Get photo filenames from google drive
    from google.colab import drive

    gdrive = "/content/gdrive/"
    drive.mount(gdrive, force_remount=True)

    # Path to folder of original photos
    basename = Path(f"{gdrive}/MyDrive/photos_child")

else:
    # Path to local folder
    basename = Path("c:/Users/brend/desktop/cartertimelapse/photos_child/")

In [ ]:
# Collect file names of original photos
filenames = sorted(basename.glob(file_identifier))
n_files = len(filenames)
print(f"{n_files} images were found, representing {n_files / 365.25:.2f} years.")

In [ ]:
# Create output folder for aligned images
out_dir = basename.parent / f"{basename.name}_aligned"
if not out_dir.exists():
    out_dir.mkdir(parents=True, exist_ok=True)

## 1.4. Identify duplicates and missing dates

Based on the filenames, let's identify missing photos and duplicates.

In [ ]:
# Collect dates from the filenames
dates = [f.name.split("PXL_")[1][:8] for f in filenames]

In [ ]:
# List all duplicates
duplicate_dates = pd.Series(dates).value_counts()
duplicates = duplicate_dates[duplicate_dates.gt(1)].sort_index().index.values
print(f"Duplicates were found for the following dates:\n{duplicates}")
print("Please make sure to only have one entry per date.")

Next, let's identify missing dates.

In [ ]:
# Convert to pandas DateTime
dates_pd = pd.to_datetime(dates, format="%Y%m%d")

# Generate a complete date range
date_range = pd.date_range(start=dates_pd.min(), end=dates_pd.max())

# Identify missing dates by finding dates in the complete range not in the original series
missing_dates = date_range.difference(dates_pd)

print(f"Missing dates can be found for the following dates:\n{missing_dates}")

Once all duplicates were removed (or renamed) and the missing dates were filled, you are good to move on to the next section.

# 2. Face alignment

Automatic face detection (with additional manual correction) using `dlib` and `skimage`.

In [ ]:
if IN_COLAB:
    dlib_path = "boy_ages/dlib_dat"
else:
    dlib_path = "dlib_dat"

# Load dlib models for face recognition
shape_predictor = dlib.shape_predictor(f"{dlib_path}/shape_predictor_5_face_landmarks.dat")  # Faces landmarks (points)

# Define which face detector to use
hog_detector = dlib.get_frontal_face_detector()
cnn_detector = dlib.cnn_face_detection_model_v1(f"{dlib_path}/mmod_human_face_detector.dat")

## 2.1. Face detection - pass 1: Using `hog_detector`

In [ ]:
# CNN is more advanced but takes longer; hog misses ~10% faces in total
face_detector = hog_detector

In [ ]:
# Loop through all images
for fdx, f in enumerate(tqdm(filenames)):
    # Specify output filename
    out_filename = out_dir / f.name

    # Skip image processing if output file was already created
    if out_filename.exists() or (out_dir / f"PXL_{dates[fdx]}x.jpg").exists():
        continue

    # Load image
    im = io.imread(f)[..., :3]

    # Get information about image size
    w, h = im.shape[:2]
    w_offset = (max_size - w) // 2
    h_offset = (max_size - h) // 2

    # Center image in a canvas
    canvas = np.zeros((max_size, max_size, 3)).astype("uint8")
    canvas[w_offset : w_offset + w, h_offset : h_offset + h, :] = im

    # Detect faces and align image
    rectangles = [x if isinstance(x, dlib.rectangle) else x.rect for x in face_detector(canvas, 1)]
    if len(rectangles):
        landmarks = [shape_predictor(canvas, r) for r in rectangles]
        face_chips = [dlib.get_face_chip(canvas, l, size=max_size // downsample, padding=1) for l in landmarks]

        # Save aligned image
        io.imsave(out_filename, face_chips[0])

    else:
        print("No face found in", f)
        
        # Rescale canvas
        canvas_rescaled = transform.rescale(canvas, 1./downsample, anti_aliasing=True, channel_axis=-1)
        canvas_rescaled = (canvas_rescaled * 255).astype('uint8')

        # Store image to out_folder
        new_out_filename = out_dir / f"PXL_{dates[fdx]}x_no-face-found.jpg"
        io.imsave(new_out_filename, canvas_rescaled)

## 2.2. Manual correction - pass 1

Have a look at the files in the `..._aligned` folder. In those cases where the face wasn't detected or the rotation was wrong, delete the file. The code below will then try to detect the face with a more advanced algorithm.

## 2.3. Face detection - pass 2: Using `cnn_detector`

In [ ]:
# CNN is more advanced but takes much longer
face_detector = cnn_detector

In [ ]:
# Loop through all images
for fdx, f in enumerate(tqdm(filenames)):
    # Specify output filename
    out_filename = out_dir / f.name

    # Skip image processing if output file was already created
    if out_filename.exists() or (out_dir / f"PXL_{dates[fdx]}x.jpg").exists():
        continue

    # Load image
    im = io.imread(f)[..., :3]

    # Get information about image size
    w, h = im.shape[:2]
    w_offset = (max_size - w) // 2
    h_offset = (max_size - h) // 2

    # Center image in a canvas
    canvas = np.zeros((max_size, max_size, 3)).astype("uint8")
    canvas[w_offset : w_offset + w, h_offset : h_offset + h, :] = im

    # Detect faces and align image
    rectangles = [x if isinstance(x, dlib.rectangle) else x.rect for x in face_detector(canvas, 1)]
    if len(rectangles):
        landmarks = [shape_predictor(canvas, r) for r in rectangles]
        face_chips = [dlib.get_face_chip(canvas, l, size=max_size // downsample, padding=1) for l in landmarks]

        # Save aligned image
        io.imsave(out_filename, face_chips[0])

    else:
        print("Still no faces found in", f)
        
        # Rescale canvas
        canvas_rescaled = transform.rescale(canvas, 1./downsample, anti_aliasing=True, channel_axis=-1)
        canvas_rescaled = (canvas_rescaled * 255).astype('uint8')

        # Store image to out_folder
        new_out_filename = out_dir / f"PXL_{dates[fdx]}x_no-face-found.jpg"
        io.imsave(new_out_filename, canvas_rescaled)

## 2.4. Identify duplicates and missing dates in x.jpg images

Same as 1.4. but this time looking only at PXL_YYYYMMDDx.jpg images

In [ ]:
# Update the file_identifier
file_identifier = "PXL_*x.jpg"

In [ ]:
# Collect all files with the correct naming
filenames = sorted(out_dir.glob(file_identifier))

In [ ]:
# Collect dates from the filenames
dates = [f.name.split("PXL_")[1][:8] for f in filenames]

In [ ]:
# List all duplicates
duplicate_dates = pd.Series(dates).value_counts()
duplicates = duplicate_dates[duplicate_dates.gt(1)].sort_index().index.values
print(f"Duplicates were found for the following dates:\n{duplicates}")
print("Please make sure to only have one entry per date.")

Next, let's identify missing dates.

In [ ]:
# Convert to pandas DateTime
dates_pd = pd.to_datetime(dates, format="%Y%m%d")

# Generate a complete date range
date_range = pd.date_range(start=dates_pd.min(), end=dates_pd.max())

# Identify missing dates by finding dates in the complete range not in the original series
missing_dates = date_range.difference(dates_pd)

print(f"Missing dates can be found for the following dates:\n{missing_dates}")

Once all duplicates were removed (or renamed) and the missing dates were filled, you are good to move on to the next section.

## 2.5. Manual correction - pass 2

For those faces where the alignment didn't work at all, or not good enough, you will need to perform a manual alignment. So have another look at the images in the `..._aligned` folder and delete the ones that were not ok or good enough.

After this step, you will need to have all photos aligned and stored in the `..._aligned` folder, so that the video can be generated. So let's get some information about which images are still missing.

In [ ]:
# Number of images
print(f"Number of original images: {len(filenames)}")
imgs_aligned = sorted(out_dir.glob(file_identifier))
print(f"Number of aligned images:  {len(imgs_aligned)}")

To help with the alignment of the missing or misaligned images, lets' create an average image that can be used as a template.

### Create average image

In [ ]:
# Collect all images
img_avg = np.zeros_like(io.imread(imgs_aligned[0])[..., :3], dtype='float32')
for path_align in tqdm(imgs_aligned):
    img_avg += io.imread(path_align)[..., :3]

# Scale data to be within 0 to 1 range
img_avg = img_avg / len(imgs_aligned) / 255.0

In [ ]:
# Plot average images
plt.figure(figsize=(7, 7))
plt.imshow(img_avg)
plt.axis("off")
plt.show()

In [ ]:
# Store average images to file
plt.imsave(out_dir.parent / "img_01_avg.png", img_avg)

### Create mosaic plots

In [ ]:
# Plot a grid of certain size of first x images
grid_size = np.array([5, 3])
scale_factor = 4
grid_points = np.prod(grid_size)
imgs_grid = np.array_split(imgs_aligned, grid_points)
fig, axes = plt.subplots(grid_size[1], grid_size[0], figsize=grid_size * scale_factor, facecolor=(0, 0, 0))
for i, ax in tqdm(zip(np.arange(grid_points), axes.flatten()), total=grid_points):
    
    # Create average image
    imgs = imgs_grid[i]
    img_avg = np.zeros_like(io.imread(imgs[0])[..., :3], dtype='float32')
    for path_align in imgs:
        img_avg += io.imread(path_align)[..., :3]

    # Scale data to be within 0 to 1 range
    img_avg = np.clip(img_avg / len(imgs), 0, 255)

    ax.imshow(img_avg.astype("int"))
    ax.axis("off")
plt.tight_layout()
plt.savefig(out_dir.parent / "img_02_mosaic.png")
del fig

In [ ]:
# Plot a grid of certain size of first x images
grid_size = np.array([5, 3])
scale_factor = 4
grid_points = np.prod(grid_size)
imgs_grid = np.array_split(imgs_aligned, grid_points)
fig, axes = plt.subplots(grid_size[1], grid_size[0], figsize=grid_size * scale_factor, facecolor=(0, 0, 0))
counter = 0
for i, ax in tqdm(zip(np.arange(grid_points), axes.flatten()), total=grid_points):

    n_images = len(imgs_grid[i])
    date_str = dates[counter + n_images // 2]
    title = f"{date_str[6:]} / {date_str[4:6]} / {date_str[:4]}"

    # Create average image
    imgs = imgs_grid[i]
    img_avg = np.zeros_like(io.imread(imgs[0])[..., :3], dtype='float32')
    for path_align in imgs:
        img_avg += io.imread(path_align)[..., :3]

    # Scale data to be within 0 to 1 range
    img_avg = np.clip(img_avg / len(imgs), 0, 255)

    ax.imshow(img_avg.astype("int"))              
    ax.set_title(title, c="white")
    ax.axis("off")
    counter += n_images
plt.tight_layout()
plt.savefig(out_dir.parent / "img_02_mosaic_date.png")
del fig

# 3.1. Video 1: No averaging

The first video is a simple stacking of all images and each image is given an index number. Note: The font for the text in the image was taken from [here](https://fonts.google.com/).

In [ ]:
# Specify frames per second
fps = 30

# Extract number of images
N_total = len(imgs_aligned)

print("Video length: %.2f seconds." % (N_total / fps))

In [ ]:
# Save temporary folder to store images to disk
tmp_dir = Path(out_dir / "tmp_video_imgs")
if not tmp_dir.exists():
    tmp_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
# Collect images with text
filenames_img = []

# Loop through all images and add text
for idx in tqdm(np.arange(len(imgs_aligned))):

    # Load aligned image
    im = io.imread(imgs_aligned[idx])
    
    # Establish out_filename
    out_filename_img = tmp_dir / f"{idx+1:05d}_image.jpg"
    out_filename_txt = tmp_dir / f"{idx+1:05d}_text.jpg"

    # Store image to file
    io.imsave(out_filename_img, np.array(im))

    # Collect year specific information
    n_year = idx / 365.25
    n_month = n_year * 12 % 12
    n_days = idx % 31
    n_year = int(np.floor(n_year))
    n_month = int(np.floor(n_month))
    n_days = int(np.floor(n_days))

    # Establish text on image
    date_str = dates[idx]
    title_txt = f"{date_str[4:6]} / {date_str[6:]} / {date_str[:4]} - Day #{idx+1:05d}\n"
    title_txt += f"{n_year:02d} years, {n_month:02d} months, {n_days+1:02d} days"
    
    # Add text to output filename
    img_txt = Image.fromarray(im)
    draw = ImageDraw.Draw(img_txt)
    width = max_size // downsample
    font_path = ""
    if IN_COLAB:
        font_path = "boy_ages/"

    # Set font, font size and x/y location of top left corner
    font_size = 0.06
    xx, yy = width * 0.13, width * 0.86

    # Draw background for text field
    text_size = (1650, 250)
    padding = 10  # Padding around text
    draw.rectangle([xx - padding, yy - padding, xx + text_size[0] + padding,
                    yy + text_size[1] + padding*4], fill=(16, 16, 16, 128))
    
    # Write text to image
    font = ImageFont.truetype(f"{font_path}Roboto/Roboto-Light.ttf", int(width * font_size))
    draw.text((xx, yy), title_txt, (255, 255, 255), font=font)

    # Store image with text to file
    io.imsave(out_filename_txt, np.array(img_txt))

    # Add image with text to list
    filenames_img.append(out_filename_img)

del img_txt

In [ ]:
# Define name of tmpd dir and filename of video 1
tmp_dir_name = str(tmp_dir)
video_name = str(out_dir.parent / "video_01_aligned.mp4")

In [ ]:
# Takes all images from tmp-folder and creates a video out of it with ffmpeg
!ffmpeg -y -framerate $fps -i "$tmp_dir_name\%05d_text.jpg" -vcodec mjpeg -vcodec libx264 $video_name
#!cat $tmp_dir_name/*_text.jpg | ffmpeg -y -f image2pipe -r $fps -vcodec mjpeg -i - -vcodec libx264 $video_name

# 3.2. Video 2: Smoothed / averaged images

The second video takes multiple images at once, averages them and then stacks them, leading to a temporal smoothing effect. Note: The font for the text in the image was taken from [here](https://fonts.google.com/).

In [ ]:
# Specify frames per second
fps = 30

In [ ]:
# Decide for mean or median averaging
use_mean = True
if use_mean:
    suffix = "mean"
else:
    suffix = "median"

### Adaptive smoothing and fps

The following parameter guides the adaptive smoothing / averaging and dynamic fps routine. Take for example:

```python
smooth_dict = {0: [7, 3],
               14: [14, 15],
               90: [30, 30],
               1095: [60, 30]}
```

This code means that from photo 0 (in Python that's the 1st one) to photo 14, the average is created over the last 7 days and this period uses an fps of 3 images/second (=3 days/second). Then from photo 14 to 90, images are averaged over last 14 days with an fps of 15. From photo 90 till 1095 (= 3 years) images are averaged over last 30 images with an fps of 30. And after that the average is done over the last 60 days, with an fps of 30.

Depending on how you set `smooth_dict` you can have different kinds of videos:

**Option 1**
```python
smooth_dict = {0: [30, 30]}
```
This will average each image over last 30 days and uses a constant fps of 30.

**Option 2**
```python
smooth_dict = {0: [7, 30],
               14: [14, 30],
               90: [30, 30],
               1095: [60, 30]}
```
This will have different averages. First 14 days only average over 7 days, then over 14 days, ...

**Option 3**
```python
smooth_dict = {0: [7, 3],
               14: [14, 15],
               90: [30, 30],
               1095: [60, 30]}
```
This changes days to average and fps.

**Option 4**
```python
smooth_dict = {0: [1, 1],
               30: [30, 30]}
```
This means that the first 30 days are not averaged at all and each image is shown for 1 second.

**Option 5**
```python
# Linearly increase the frame rate to 30 fps, for the first 50 images
linear_increase = np.linspace(1, 30, num=50, dtype='int')
smooth_dict = dict([[idx, [1, linear_increase[idx]]] for idx in np.arange(50)])

# At 30 fps, linearly increase the averaging of images from 1 to 30 for images 50 to 200
offset = 50
linear_increase = np.linspace(1, 30, num=150, dtype='int')
smooth_dict.update(dict([[idx + offset, [linear_increase[idx], 30]] for idx in np.arange(150)]))

# Keep constant average of 30 with 30 fps, from image 200 on
smooth_dict[200]= [30, 30]
```
This means that the first 50 days will not be averaged (hence the `1` in `[1, linear_increase[idx]]`), and will gradually speed up the frame rate to 30 fps, over the first 50 days. After that it will gradually speed up the number of images it afterages for 150 days (i.e. day 50 to 200). And lastly, from day 200 on, it will keep constant averaging and fps of 30.

In [ ]:
# Linearly increase the frame rate to 30 fps, for the first 50 images
linear_increase = np.linspace(1, 30, num=50, dtype='int')
smooth_dict = dict([[idx, [1, linear_increase[idx]]] for idx in np.arange(50)])

# At 30 fps, linearly increase the averaging of images from 1 to 30 for images 50 to 200
offset = 50
linear_increase = np.linspace(1, 30, num=150, dtype='int')
smooth_dict.update(dict([[idx + offset, [linear_increase[idx], 30]] for idx in np.arange(150)]))

# Keep constant average of 30 with 30 fps, from image 200 on
smooth_dict[200]= [30, 30]

In [ ]:
# Loop through images and average them according to smooth_dict
idx = 0
fdx = 0
smooth = 1

n_images = len(filenames_img)
pbar = tqdm()
while True:
    
    # Ignore loop if all images were processed
    if idx >= len(filenames_img) and np.std(imgs_idx) == 0:
        break

    # Acquire temporal smoothing length and fps duplication factor
    if idx in smooth_dict.keys():
        smooth, fps_fact = smooth_dict[idx]

    # Collect indeces of images
    imgs_idx = np.clip(np.arange(idx - smooth, idx) + 1, 0, n_images - 1)

    # Collect images
    imgs_group = np.array([io.imread(filenames_img[loc])[..., :3] for loc in imgs_idx])
    
    # Average images
    if use_mean:
        img_comp = np.mean(imgs_group, axis=0).astype("uint8")
    else:
        img_comp = np.median(imgs_group, axis=0).astype("uint8")

    # Establish date string
    date_str = np.array(dates)[imgs_idx]
    date_str = date_str[len(date_str)//2]
        
    # Establish text on image
    title_txt = f"{date_str[4:6]} / {date_str[6:]} / {date_str[:4]} - Day #{idx+1:05d}\n"
    title_txt += f"{n_year:02d} years, {n_month:02d} months, {n_days+1:02d} days"
    
    # Add text to output filename
    img_txt = Image.fromarray(img_comp)
    draw = ImageDraw.Draw(img_txt)
    width = max_size // downsample
    font_path = ""
    if IN_COLAB:
        font_path = "boy_ages/"

    # Set font, font size and x/y location of top left corner
    font_size = 0.06
    xx, yy = width * 0.13, width * 0.86

    # Draw background for text field
    text_size = (1650, 250)
    padding = 10  # Padding around text
    draw.rectangle([xx - padding, yy - padding, xx + text_size[0] + padding,
                    yy + text_size[1] + padding*4], fill=(16, 16, 16, 128))
    
    # Write text to image
    font = ImageFont.truetype(f"{font_path}Roboto/Roboto-Light.ttf", int(width * font_size))
    draw.text((xx, yy), title_txt, (255, 255, 255), font=font)
        
    # Create output file and duplicate it if necessairy (to decrease fps)
    duplication_factor = np.maximum(int(fps / fps_fact), 1)  # needs to be at least 1
    for _ in range(duplication_factor):
        # Save composition image
        out_filename = tmp_dir / f"{fdx+1:05d}_{suffix}.jpg"
        io.imsave(out_filename, np.array(img_txt).astype("uint8"))
        fdx += 1

    # Report progress
    if idx % 30 == 0:
        ddx = np.minimum(idx, len(dates) - 1)
        print(f"Finished date {dates[ddx]}")

    # Increase indecies
    idx += 1

    # Update the progress bar
    pbar.update(1)

# Close the progress bar
pbar.close()

In [ ]:
# Define name of tmpd dir and filename of video 2
tmp_dir_name = str(tmp_dir)
video_name = str(out_dir.parent / f"video_02_smoothed_{suffix}-{smooth}-days.mp4")

In [ ]:
# Takes all images from tmp-folder and creates a video out of it with ffmpeg
!ffmpeg -y -framerate $fps -i "$tmp_dir_name\%05d_{suffix}.jpg" -vcodec mjpeg -vcodec libx264 $video_name
#!cat $tmp_dir_name/*jpg | ffmpeg -y -f image2pipe -r $fps -vcodec mjpeg -i - -vcodec libx264 $video_name

## 3.3 Clean up

Remove all jpg files in the temporary directory that was used to create the frames for the video.

In [ ]:
# Remove all temporary files
for p in sorted(tmp_dir.glob("*jpg")):
    p.unlink()